# 图和对话
TensorFlow 使用数据流图将计算表示为独立的指令之间的依赖关系。

可以通过低级的api构建编程模型，在该模型中：

首先定义数据流图，然后创建 TensorFlow 会话，以便在一组本地和远程设备上运行图的各个部分。

较高阶的 API（例如 tf.estimator.Estimator 和 Keras）会向最终用户隐去图和会话的细节内容。

使用低级API可以更好了解工作细节。

## 图 
***一般而言，使用TensorFlow程序的流程是先创建一个图，然后在session中启动它。***

计算图是排列成一个图的一系列 TensorFlow 指令。图由两种类型的对象组成。

- 操作（简称“op”）：图的节点。操作描述了消耗和生成张量的计算。
- 张量：图的边。它们代表将流经图的值。大多数 TensorFlow 函数会返回 tf.Tensors。

我们来构建一个简单的计算图。<br>
a与b是一行常量的张量

a + b 就是一个操作

他们在一起组成一个简单的计算图

In [19]:
import tensorflow as tf 

a = tf.constant(3.0, dtype=tf.float32)
b = tf.constant(4.0) # also tf.float32 implicitly
total = a + b
print(a)
print(b)
print(total)

Tensor("Const_4:0", shape=(), dtype=float32)
Tensor("Const_5:0", shape=(), dtype=float32)
Tensor("add_3:0", shape=(), dtype=float32)


**注意**，打印张量并不会如预期的那样输出值 3.0、4.0 和 7.0。

上述语句只会构建计算图。这些 tf.Tensor 对象仅代表将要运行的操作的结果。

图中的每个指令都拥有唯一的名称。这个名称不同于使用 Python 分配给相应对象的名称。

张量是根据生成它们的指令命名的，后面跟着输出索引，如 "add:0"。

TensorFlow Python库已经有一个默认图 (default graph)，如果没有创建新的计算图，则默认情况下是在这个default graph里面创建节点和边。

现在默认图就有了三个节点，两个constant()，和一个add()。

为了真正使两个张量相加并得到结果，就必须在会话里面启动这个图。

## 会话（Session）
要启动计算图(或者说要获得（运行）张量)，首先要创建一个Session对象。

会话会封装 TensorFlow 运行时的状态，并运行 TensorFlow 操作。

如果说 tf.Graph 像一个 .py 文件，那么 tf.Session 就像一个 python 可执行对象。

使用tf.Session()创建会话，调用run()函数执行计算图。

如果没有传入任何创建参数，会话构造器将启动默认图。

如果要指定某个计算图，则传入计算图参数（如 图g1），则创建会话方式为tf.Session(graph=g1)。

### 创建会话（Session）
主要有以下2种方式：

In [20]:
# 方式1
#启动默认图
sess=tf.Session()
res = sess.run(total)
print(res)

# 任务完成, 关闭会话.
sess.close()
# Session在使用完后需要关闭以释放资源，除了显式调用close外，
# 也可以使用“with”代码块 来自动完成关闭动作（一般都会在with中完成）。

7.0


In [21]:
# 方式2
with tf.Session() as sess:
    res = sess.run(total)
    print(res)

7.0


当使用 Session.run 请求输出节点时，TensorFlow 会回溯整个图

并流经提供了所请求的输出节点对应的输入值的所有节点。因此此指令会打印预期的值 7.0

可以将多个张量传递给 tf.Session.run。run 方法以透明方式处理元组或字典的任何组合，如下例所示：

In [22]:
with tf.Session() as sess:
    print(sess.run({'ab':(a, b), 'total':total}))

{'ab': (3.0, 4.0), 'total': 7.0}


在调用 tf.Session.run 期间，任何 tf.Tensor 都只有单个值。<br>
例如，以下代码调用 tf.random_uniform 来生成一个 tf.Tensor，后者会生成随机的三元素矢量（值位于 [0,1) 区间内）。<br>
每次调用 run 时，结果都会显示不同的随机值，但在单个 run 期间（out1 和 out2 接收到相同的随机输入值），结果显示的值是一致的：

In [28]:
vec = tf.random_uniform(shape=(3,))
out1 = vec + 1
out2 = vec + 2
with tf.Session() as sess:
    print(sess.run(vec))
    print(sess.run(vec))
    print(sess.run((out1, out2)))

[0.07510519 0.06543934 0.18779802]
[0.13146317 0.05826223 0.14126337]
(array([1.0011921, 1.3317279, 1.1881571], dtype=float32), array([2.001192, 2.331728, 2.188157], dtype=float32))


### Feeding
目前的图只能使用常量值，因此它总是生成一个常量结果。<br>
图可以参数化以便接受外部输入，也称为占位符。占位符表示在稍后提供值，它就像函数参数。

In [31]:
x = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32)
z = x + y

前面三行有点像函数。我们定义了这个函数的两个输入参数（x 和 y），然后对它们指令。

我们可以使用 run 方法的 feed_dict 参数为占位符提供具体的值，从而评估这个具有多个输入的图：

In [33]:
with tf.Session() as sess:
    print(sess.run(z, feed_dict={x: 3, y: 4.5}))
    print(sess.run(z, feed_dict={x: [1, 3], y: [2, 4]}))

7.5
[3. 7.]


### 默认的会话
请看`6_default_session`

### 交互式session
请看`4_Interactivesession`。